In [7]:
import torch
import gradientUtils as gu
# Define a threshold for early stopping
threshold = 1e-16


### AUTO GRAD

In [8]:
def autograd():    
    # Define the ground truth vertex
    X_g = torch.tensor([1.0], requires_grad=False)
    Y_g = torch.tensor([1.0], requires_grad=False)
    # Initialize the sites
    s_i = gu.Site(1.0, 1.0)
    s_j = gu.Site(2.0, 2.0)
    s_k = gu.Site(1.50, 2.750)
    # Define an optimizer
    optimizer = torch.optim.Adam([s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y], lr=0.01)
    # Previous loss
    prev_loss = float('inf')
    # Training loop
    for epoch in range(1000):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Compute the vertex
        X, Y = gu.compute_vertex(s_i, s_j, s_k)
        
        # Compute the loss
        loss = (X - X_g)**2 + (Y - Y_g)**2
        # Early stopping condition
        if abs(prev_loss - loss.item()) < threshold:
            print(f"Stopping early at epoch {epoch} due to minimal loss change")
            gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g,"images/autograd/")
            break
    
        prev_loss = loss.item()
        # Backpropagate the error
        loss.backward()
        
        # Update the parameters
        optimizer.step()
        
        # Print the loss
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')
            gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g,"images/autograd/")
    
    # Print the final site coordinates
    print(f'Final site coordinates:')
    print(f's_i: ({s_i.x.item()}, {s_i.y.item()})')
    print(f's_j: ({s_j.x.item()}, {s_j.y.item()})')
    print(f's_k: ({s_k.x.item()}, {s_k.y.item()})')
    
    # Compute the final vertex
    final_X, final_Y = gu.compute_vertex(s_i, s_j, s_k)
    
    # Print the final vertex coordinates
    print(f'Final vertex coordinates: ({final_X.item()}, {final_Y.item()})')
    return s_i, s_j, s_k

s_i, s_j, s_k = autograd()

Epoch 0, Loss: 0.8612499237060547
Epoch 100, Loss: 0.0021477900445461273
Epoch 200, Loss: 5.845098627332845e-09
Stopping early at epoch 279 due to minimal loss change
Final site coordinates:
s_i: (0.10342850536108017, 0.35356053709983826)
s_j: (1.8965137004852295, 1.6465104818344116)
s_k: (1.1535061597824097, 2.094599962234497)
Final vertex coordinates: (0.9999982714653015, 0.9999975562095642)


### Custom gradient

In [9]:
def customgrad():    
    # Define the ground truth vertex
    X_g = torch.tensor([1.0], requires_grad=False)
    Y_g = torch.tensor([1.0], requires_grad=False)
    # Initialize the sites
    s_i = gu.Site(1.0, 1.0)
    s_j = gu.Site(2.0, 2.0)
    s_k = gu.Site(1.50, 2.750)
    # Define an optimizer
    optimizer = torch.optim.Adam([s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y], lr=0.01)
    prev_loss = float('inf')
    
    class CustomVertexFunction(torch.autograd.Function):
        @staticmethod
        def forward(ctx, x_i, y_i, x_j, y_j, x_k, y_k):
            N_X = x_i**2 * (y_j - y_k) - x_j**2 * (y_i - y_k) + (x_k**2 + (y_i - y_k) * (y_j - y_k)) * (y_i - y_j)
            N_Y = -(x_i**2 * (x_j - x_k) - x_i * (x_j**2 - x_k**2 + y_j**2 - y_k**2) + x_j**2 * x_k - x_j * (x_k**2 - y_i**2 + y_k**2) - x_k * (y_i**2 - y_j**2))
            D = 2 * (x_i * (y_j - y_k) - x_j * (y_i - y_k) + x_k * (y_i - y_j))
            
            X = N_X / D
            Y = N_Y / D
            
            ctx.save_for_backward(x_i, y_i, x_j, y_j, x_k, y_k, X, Y, N_X, N_Y, D)
            return X, Y
    
        @staticmethod
        def backward(ctx, grad_output_X, grad_output_Y):
            x_i, y_i, x_j, y_j, x_k, y_k, X, Y, N_X, N_Y, D = ctx.saved_tensors
            
            # Compute partial derivatives of X and Y wrt each site coordinate
            dN_X_dx_i = 2 * x_i * (y_j - y_k)
            dN_X_dx_j = -2 * x_j * (y_i - y_k)
            dN_X_dx_k = 2 * x_k * (y_i - y_j)
            
            dN_Y_dx_i = -2 * x_i * (x_j - x_k) - x_j**2 + x_k**2 - y_j**2 + y_k**2
            dN_Y_dx_j = -x_i**2 + x_k**2
            dN_Y_dx_k = -x_i**2 + x_j**2
            
            dD_dx_i = 2 * (y_j - y_k)
            dD_dx_j = 2 * (y_i - y_k)
            dD_dx_k = 2 * (y_i - y_j)
            
            dX_dx_i = (dN_X_dx_i * D - N_X * dD_dx_i) / (D**2)
            dX_dx_j = (dN_X_dx_j * D - N_X * dD_dx_j) / (D**2)
            dX_dx_k = (dN_X_dx_k * D - N_X * dD_dx_k) / (D**2)
            
            dY_dx_i = (dN_Y_dx_i * D - N_Y * dD_dx_i) / (D**2)
            dY_dx_j = (dN_Y_dx_j * D - N_Y * dD_dx_j) / (D**2)
            dY_dx_k = (dN_Y_dx_k * D - N_Y * dD_dx_k) / (D**2)
            
            # Similarly for the y coordinates
            dN_X_dy_i = 2 * (x_j - x_k) * x_i - x_j**2 + x_k**2 + y_j**2 - y_k**2
            dN_X_dy_j = -2 * (x_i - x_k) * x_j + x_i**2 - x_k**2 - y_i**2 + y_k**2
            dN_X_dy_k = 2 * (x_i - x_j) * x_k - x_i**2 + x_j**2 + y_i**2 - y_j**2
            
            dN_Y_dy_i = 0
            dN_Y_dy_j = 0
            dN_Y_dy_k = 0
            
            dD_dy_i = 2 * (x_j - x_k)
            dD_dy_j = 2 * (x_i - x_k)
            dD_dy_k = 2 * (x_i - x_j)
            
            dX_dy_i = (dN_X_dy_i * D - N_X * dD_dy_i) / (D**2)
            dX_dy_j = (dN_X_dy_j * D - N_X * dD_dy_j) / (D**2)
            dX_dy_k = (dN_X_dy_k * D - N_X * dD_dy_k) / (D**2)
            
            dY_dy_i = (dN_Y_dy_i * D - N_Y * dD_dy_i) / (D**2)
            dY_dy_j = (dN_Y_dy_j * D - N_Y * dD_dy_j) / (D**2)
            dY_dy_k = (dN_Y_dy_k * D - N_Y * dD_dy_k) / (D**2)
            
            grad_x_i = grad_output_X * dX_dx_i + grad_output_Y * dY_dx_i
            grad_y_i = grad_output_X * dX_dy_i + grad_output_Y * dY_dy_i
            grad_x_j = grad_output_X * dX_dx_j + grad_output_Y * dY_dx_j
            grad_y_j = grad_output_X * dX_dy_j + grad_output_Y * dY_dy_j
            grad_x_k = grad_output_X * dX_dx_k + grad_output_Y * dY_dx_k
            grad_y_k = grad_output_X * dX_dy_k + grad_output_Y * dY_dy_k
            
            return grad_x_i, grad_y_i, grad_x_j, grad_y_j, grad_x_k, grad_y_k
    
    def compute_vertex(s_i, s_j, s_k):
        return CustomVertexFunction.apply(s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y)
    
    
    # Training loop
    for epoch in range(1000):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Compute the vertex
        X, Y = compute_vertex(s_i, s_j, s_k)
        
        # Compute the loss
        loss = (X - X_g)**2 + (Y - Y_g)**2
        # Early stopping condition
        if abs(prev_loss - loss.item()) < threshold:
            print(f"Stopping early at epoch {epoch} due to minimal loss change")
            gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g,"images/customgrad/")
            break
    
        prev_loss = loss.item()
        # Backpropagate the error
        loss.backward()
        
        # Update the parameters
        optimizer.step()
        
        # Print the loss
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')
            gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g,"images/customgrad/")
    
    # Print the final site coordinates
    print(f'Final site coordinates:')
    print(f's_i: ({s_i.x.item()}, {s_i.y.item()})')
    print(f's_j: ({s_j.x.item()}, {s_j.y.item()})')
    print(f's_k: ({s_k.x.item()}, {s_k.y.item()})')
    
    # Compute the final vertex
    final_X, final_Y = compute_vertex(s_i, s_j, s_k)
    
    # Print the final vertex coordinates
    print(f'Final vertex coordinates: ({final_X.item()}, {final_Y.item()})')
    return s_i, s_j, s_k
    
s_i, s_j, s_k = customgrad()


Epoch 0, Loss: 0.8612499237060547
Epoch 100, Loss: 0.16594450175762177
Epoch 200, Loss: 0.0030409719329327345
Epoch 300, Loss: 4.169791282038204e-06
Stopping early at epoch 381 due to minimal loss change
Final site coordinates:
s_i: (-0.010905168950557709, 1.029019832611084)
s_j: (1.496026635169983, 1.881323218345642)
s_k: (0.9567925333976746, 2.0104000568389893)
Final vertex coordinates: (0.9999986290931702, 1.0000028610229492)
